# Import

In [19]:
data_path='data/'
api_key='ehlVSIKSMiugJZHlii3sU7OKCexe4MIlnYenGric'
import pandas as pd 

# Analysis

In [20]:
df = pd.read_csv(f"{data_path}epi_r.csv")
df_uniq=pd.DataFrame({
    'unique_values': df.nunique()
})
for i in df.columns:
    print(i, ":", df_uniq.loc[i].item())

title : 17736
rating : 8
calories : 1858
protein : 282
fat : 326
sodium : 2434
#cakeweek : 2
#wasteless : 2
22-minute meals : 2
3-ingredient recipes : 2
30 days of groceries : 2
advance prep required : 2
alabama : 2
alaska : 2
alcoholic : 2
almond : 2
amaretto : 2
anchovy : 2
anise : 2
anniversary : 2
anthony bourdain : 2
aperitif : 2
appetizer : 2
apple : 2
apple juice : 2
apricot : 2
arizona : 2
artichoke : 2
arugula : 2
asian pear : 2
asparagus : 2
aspen : 2
atlanta : 2
australia : 2
avocado : 2
back to school : 2
backyard bbq : 2
bacon : 2
bake : 2
banana : 2
barley : 2
basil : 2
bass : 2
bastille day : 2
bean : 2
beef : 2
beef rib : 2
beef shank : 2
beef tenderloin : 2
beer : 2
beet : 2
bell pepper : 2
berry : 2
beverly hills : 2
birthday : 2
biscuit : 2
bitters : 2
blackberry : 2
blender : 2
blue cheese : 2
blueberry : 2
boil : 2
bok choy : 2
bon appétit : 2
bon app��tit : 2
boston : 2
bourbon : 2
braise : 2
bran : 2
brandy : 2
bread : 2
breadcrumbs : 2
breakfast : 2
brie : 2
bri

The dataset have 2 mandatory features:
```title``` and ```rating```.

There is also 4 optional features: calories, protein, fat, sodium 



Dataset have 2 categorical features after the One-Hot Encoding conversion.

first categorical feature is:

Ingredients: ```almond, anchovy, apple, avocado, bacon```

second categorical feature is tags

The tags may contain:
* Recipe characteristics: ```#cakeweek, #wasteless, 22-minute meals, 3-ingredient recipes...```

* Geographical affiliation: ```alabama, arizona, boston, france, italy...```

* Dietary restrictions and features: ```dairy free, gluten free, vegan...```

* Cooking methods: ```bake, boil, grill, slow cooker...```

* Holidays and events: ```christmas, halloween, valentine's day...```

* Machinery and tools: ```blender, food processor, pressure cooker, microwave...```

## 1. We need to discuss, about optional features аre we leaving them or not

## 2. We need to sort categorical columns in the dataset (leave only the ingredients)

кароче грубо говоря нужно с помощью апи проверять является ли признак ингредиентом, если да оставляем его в выборке 

https://api.nal.usda.gov/fdc/v1/foods/search?api_key=DEMO_KEY&query=Cheddar%20Cheese

In [21]:
import requests
import json
import csv
def is_ingredients(ingredient):
    params = {
        "query": ingredient,
        "api_key": api_key,
        "dataType": ["Branded"]  
    }
    response = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search', params=params)
    if response.status_code == 200:
        json = response.json()  
        data={

        }
        return json

In [22]:
def json_to_csv(json_data, csv_file="output.csv"):
    df = pd.json_normalize(json_data)
    df.to_csv(csv_file, index=False)
json_to_csv(json_data)

Let's see which columns are ingredients 